In [4]:
from maliboo import BayesianOptimization, UtilityFunction
import numpy as np
import os

import matplotlib.pyplot as plt
from matplotlib import gridspec
%matplotlib inline

In [5]:
def target(x,y):
    #return (np.exp(-(x - 2)**2) + np.exp(-(x - 6)**2/10) + 1/ (x**2 + 1))
    #return  -x**2
    #return -np.abs(x)
    #return np.cos(x)
    #return -x**2 -y**2
    return -(x + y)

# x = np.linspace(-7, 7, 10000).reshape(-1, 1)
# y = target(x)

# plt.plot(x,y)

In [6]:
import warnings
def posterior(optimizer, x_obs, y_obs, grid):
    optimizer._gp.fit(x_obs, y_obs)
    mu, sigma = optimizer._gp.predict(grid, return_std=True)
    return mu, sigma

def plot_gp(optimizer, x, y, show_minimized=False, style='transparent'):

    if style == 'classic':
        plt.style.use(style)
    elif style == 'dark':
        plt.style.use('dark_background')
    elif style == 'transparent':
        pass
    else:
        raise ValueError("Wrong style")

    # Swap sign of stuff
    sign = -1 if show_minimized else 1
    
    # Set up complete plot
    plt.rcParams.update({'font.size': 20})

    fig = plt.figure(figsize=(16, 10))
    steps = len(optimizer.space)
    gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1]) 
    axis = plt.subplot(gs[0])
    acq = plt.subplot(gs[1])

    ## MAIN PLOT
    # Plot true objective
    target_col = 'limegreen' if style == 'dark' else 'gray'
    axis.plot(x, sign*y, linewidth=3.5, label='objective function',
              color=target_col)
    # Plot observations
    x_obs = np.array([[res["x"]] for res in optimizer.res])
    y_obs = np.array([res["target"] for res in optimizer.res])
    axis.scatter(x_obs, sign*y_obs, marker='o', s=300,
                 label='observations', color='red', zorder=10)   
    # Plot mean function
    mu, sigma = posterior(optimizer, x_obs, y_obs, x)
    mu = sign * mu
    axis.plot(x, mu, dashes=(12, 3.5), color='blue', lw=2.5, label='mean prediction')
    # Plot credible interval
    quant = 1.96
    axis.fill(np.concatenate([x, x[::-1]]), 
              np.concatenate([mu - quant * sigma, (mu + quant * sigma)[::-1]]),
        alpha=.6, fc='paleturquoise', ec='None', label='95% credible interval')
    # Set parameters
    xlim = (-2,10)
    axis.set_xlim(xlim)
    axis.set_xlabel('x')
    axis.set_ylim(-1.61, 0.01)
    axis.set_ylabel('objective function')
    axis.legend(loc=2, bbox_to_anchor=(1.01, 1), borderaxespad=0.)

    ## ACQUISITION PLOT
    utility_function = UtilityFunction(kind="ucb", kappa=5, xi=0)
    utility = utility_function.utility(x, optimizer._gp, 0)
    x_next = x[np.argmax(utility)]
    print(x_next)
    acq.plot(x, utility, label='acquisition function', lw=2, color='fuchsia')
    acq.scatter(x_next, np.max(utility), marker='x', s=300, 
             label='next observation', ec='red', lw=4, zorder=10)
    acq.set_xlim(xlim)
    acq.set_ylim((0, 2.501))
    acq.set_ylabel('utility')
    acq.set_xlabel('x')
    acq.legend(loc=2, bbox_to_anchor=(1.01, 1))

    # Vertical lines in both
    axis.vlines(x_next, sign*0, sign*1.6, lw=1.5, linestyle=(0, (10, 3.5)), color='silver')
    acq.vlines(x_next,       0,        5, lw=1.5, linestyle=(0, (10, 3.5)), color='silver')

    # Overlay vertical Gaussian distribution
    ypos_idx = np.abs(x-x_next).argmin()
    ypos = mu[ypos_idx]
    var = 0.6 * sigma[ypos_idx]  # I'm cheating a little bit with the variance
                                 # for representation purposes
    bound = 0.5
    supp = np.linspace(-bound, bound, 1000).reshape(-1, 1)
    gaussian = np.exp((-supp ** 2.0) / (2 * var ** 2.0))
    gaussian /= gaussian.max()
    gaussian *= -0.5  # height
    axis.plot(gaussian + x_next, supp + ypos, lw=1.5, color='silver')

    # Save to file
    for fmt in ('png', 'svg'):
        filename = f'bo_{style}_{len(x_obs)}.{fmt}'
        plt.savefig(os.path.join('resources', filename), bbox_inches='tight', dpi=300)

# # UCB
# # Initialize optimizer
# optimizer = BayesianOptimization(target, {'x': (-2, 10)},
#                                  random_state=272242)
# # First rounds of optimization
# optimizer.maximize(init_points=3, n_iter=1, kappa=5)
# for style in ('classic', 'dark'):  # ('transparent', 'classic', 'dark')
#     plot_gp(optimizer, x, y, show_minimized=True, style=style)

# EB
# Initialize optimizer again
optimizer = BayesianOptimization(target, {'x': (-2,2)
                                          ,'y': (-2,2)
                                           }, 
                                 random_state=27222,barrier_func={'constraint1': lambda x,y: 3/2 -x-2*y - 1/2*np.sin(2*np.pi*(x**2 - 2*y)),
                                                                  'constraint2': lambda x,y: x**2 + y**2 - 3/2
                                                                  }, debug = True)
# Other rounds of optimization
with warnings.catch_warnings():
    warnings.simplefilter("ignore", RuntimeWarning)
    warnings.simplefilter("ignore", FutureWarning)
    optimizer.maximize(init_points=10, n_iter=20,acq='eb',acq_info={'static_lambda':False})


Initializing TargetSpace with bounds: {'x': (-2, 2), 'y': (-2, 2)}
initialize_dataset(): dataset is None
TargetSpace initialization completed
BayesianOptimization initialization completed
Starting maximize()
|   iter    |  target   |     x     |     y     |
-------------------------------------------------
_prime_queue(): initializing 10 random points
Uniform randomly sampled point: value [[-1.69920866  0.10726756]]
Uniform randomly sampled point: value [[-0.69272098  1.44761031]]
Uniform randomly sampled point: value [[ 0.45016905 -0.6170141 ]]
Uniform randomly sampled point: value [[ 1.20459689 -0.18937276]]
Uniform randomly sampled point: value [[-1.48596331  0.4403889 ]]
Uniform randomly sampled point: value [[-0.34390162 -1.08688852]]
Uniform randomly sampled point: value [[0.16824596 1.13926836]]
Uniform randomly sampled point: value [[1.0842995  1.88064276]]
Uniform randomly sampled point: value [[-1.1047056  -1.34128481]]
Uniform randomly sampled point: value [[ 1.89623702 -1.7

TypeError: UtilityFunction._eb() missing 1 required positional argument: 'static_lambda'

In [ ]:
import warnings
def posterior(optimizer, x_obs, y_obs, grid):
    optimizer._gp.fit(x_obs, y_obs)
    mu, sigma = optimizer._gp.predict(grid, return_std=True)
    return mu, sigma

def plot_gp(optimizer, x, y, show_minimized=False, style='transparent'):

    if style == 'classic':
        plt.style.use(style)
    elif style == 'dark':
        plt.style.use('dark_background')
    elif style == 'transparent':
        pass
    else:
        raise ValueError("Wrong style")

    # Swap sign of stuff
    sign = -1 if show_minimized else 1
    
    # Set up complete plot
    plt.rcParams.update({'font.size': 20})

    fig = plt.figure(figsize=(16, 10))
    steps = len(optimizer.space)
    gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1]) 
    axis = plt.subplot(gs[0])
    acq = plt.subplot(gs[1])

    ## MAIN PLOT
    # Plot true objective
    target_col = 'limegreen' if style == 'dark' else 'gray'
    axis.plot(x, sign*y, linewidth=3.5, label='objective function',
              color=target_col)
    # Plot observations
    x_obs = np.array([[res["x"]] for res in optimizer.res])
    y_obs = np.array([res["target"] for res in optimizer.res])
    axis.scatter(x_obs, sign*y_obs, marker='o', s=300,
                 label='observations', color='red', zorder=10)   
    # Plot mean function
    mu, sigma = posterior(optimizer, x_obs, y_obs, x)
    mu = sign * mu
    axis.plot(x, mu, dashes=(12, 3.5), color='blue', lw=2.5, label='mean prediction')
    # Plot credible interval
    quant = 1.96
    axis.fill(np.concatenate([x, x[::-1]]), 
              np.concatenate([mu - quant * sigma, (mu + quant * sigma)[::-1]]),
        alpha=.6, fc='paleturquoise', ec='None', label='95% credible interval')
    # Set parameters
    xlim = (-2,10)
    axis.set_xlim(xlim)
    axis.set_xlabel('x')
    axis.set_ylim(-1.61, 0.01)
    axis.set_ylabel('objective function')
    axis.legend(loc=2, bbox_to_anchor=(1.01, 1), borderaxespad=0.)

    ## ACQUISITION PLOT
    utility_function = UtilityFunction(kind="ucb", kappa=5, xi=0)
    utility = utility_function.utility(x, optimizer._gp, 0)
    x_next = x[np.argmax(utility)]
    print(x_next)
    acq.plot(x, utility, label='acquisition function', lw=2, color='fuchsia')
    acq.scatter(x_next, np.max(utility), marker='x', s=300, 
             label='next observation', ec='red', lw=4, zorder=10)
    acq.set_xlim(xlim)
    acq.set_ylim((0, 2.501))
    acq.set_ylabel('utility')
    acq.set_xlabel('x')
    acq.legend(loc=2, bbox_to_anchor=(1.01, 1))

    # Vertical lines in both
    axis.vlines(x_next, sign*0, sign*1.6, lw=1.5, linestyle=(0, (10, 3.5)), color='silver')
    acq.vlines(x_next,       0,        5, lw=1.5, linestyle=(0, (10, 3.5)), color='silver')

    # Overlay vertical Gaussian distribution
    ypos_idx = np.abs(x-x_next).argmin()
    ypos = mu[ypos_idx]
    var = 0.6 * sigma[ypos_idx]  # I'm cheating a little bit with the variance
                                 # for representation purposes
    bound = 0.5
    supp = np.linspace(-bound, bound, 1000).reshape(-1, 1)
    gaussian = np.exp((-supp ** 2.0) / (2 * var ** 2.0))
    gaussian /= gaussian.max()
    gaussian *= -0.5  # height
    axis.plot(gaussian + x_next, supp + ypos, lw=1.5, color='silver')

    # Save to file
    for fmt in ('png', 'svg'):
        filename = f'bo_{style}_{len(x_obs)}.{fmt}'
        plt.savefig(os.path.join('resources', filename), bbox_inches='tight', dpi=300)


optimizer = BayesianOptimization(target, {'x': (-9,9),'y': (-9,9)}, 
                                 random_state=27222,barrier_func={'constraint1': lambda x,y: x**2 -25,
                                                                  'constraint2': lambda x,y: y**2 -25
                                                                  }, debug = True)
# Other rounds of optimization
with warnings.catch_warnings():
    warnings.simplefilter("ignore", RuntimeWarning)
    warnings.simplefilter("ignore", FutureWarning)
    optimizer.maximize(init_points=10, n_iter=40,acq='eb',acq_info={'static_lambda':False})

Initializing TargetSpace with bounds: {'x': (-9, 9), 'y': (-9, 9)}
initialize_dataset(): dataset is None
TargetSpace initialization completed
BayesianOptimization initialization completed
Starting maximize()
|   iter    |  target   |     x     |     y     |
-------------------------------------------------
_prime_queue(): initializing 10 random points
Uniform randomly sampled point: value [[ 3.99922226 -6.17177636]]
Uniform randomly sampled point: value [[-4.30212095 -7.55032838]]
Uniform randomly sampled point: value [[ 0.86238846 -5.88473628]]
Uniform randomly sampled point: value [[2.09051772 8.48686164]]
Uniform randomly sampled point: value [[-6.39368149 -2.21412928]]
Uniform randomly sampled point: value [[-1.52617647 -4.91905603]]
Uniform randomly sampled point: value [[ 2.30464412 -5.48677941]]
Uniform randomly sampled point: value [[1.93590114 7.02133886]]
Uniform randomly sampled point: value [[-8.96622203 -1.14893359]]
Uniform randomly sampled point: value [[-2.22841176 -0.7

In [ ]:
optimizer.max

{'target': -0.8484848484848486,
 'params': {'x': 0.4242424242424243, 'y': 0.4242424242424243}}

In [ ]:
gp_barrier = optimizer._gps_barriers


In [ ]:
optimizer.space.x_grid

array([[-2.        , -1.95959596],
       [-1.91919192, -1.87878788],
       [-1.83838384, -1.7979798 ],
       ...,
       [ 2.        ,  2.        ],
       [ 2.        ,  2.        ],
       [ 2.        ,  2.        ]])

In [ ]:
gp_function = optimizer._gp

In [ ]:
def ac(x,gp_barrier,gp_function):
    mu_c,stdc = gp_barrier.predict(np.array([x]), return_std= True)
    mu_f,stdf = gp_function.predict(np.array([x]), return_std= True)
    return -mu_f - (stdf**2)*(np.log(-mu_c) - stdc**2/(2*mu_c**2))

In [ ]:
import scipy

In [ ]:
x_draw = np.linspace(-4,4
                     , 1000)
ac_eval = []
for i in range(len(x_draw)):
    ac_eval.append(ac([x_draw[i]],gp_barrier,gp_function))


AttributeError: 'list' object has no attribute 'predict'

In [ ]:
x = np.linspace(-10,10,10)
y = np.linspace(-10,10,10)
z = np.linspace(-10,10,10)
a = np.meshgrid(*[x,y,z])
a = np.array(a).reshape(3,-1)




In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd

# Define your 2D function
def my_function(x, y):
    # Your function implementation here
    return target(x, y)  # Make sure you have a valid implementation for target

# Define your minimization algorithm
def my_minimization_algorithm():
    out = pd.read_csv('C:\\Users\\Tommaso\\Desktop\\bproject\\MALIBOO\\results.csv')
    x = out['x'].values
    y = out['y'].values
    z = my_function(x, y) + 2
    point_list = []
    for i in range(len(x)):
        values = np.array([x[i], y[i], z[i]])
        point_list.append(values)
    return point_list

# Create a figure and axis
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot the 2D function with full transparency
X, Y = np.meshgrid(np.linspace(-10, 0, 100), np.linspace(-10, 0, 100))
Z = my_function(X, Y)
contour = ax.plot_surface(X, Y, Z, cmap="viridis", alpha=0.5)

# Plot the minimization path with transparency
path = ax.scatter([], [], [], marker='+', color='black', alpha=0.5)

def update(frame):
    points = my_minimization_algorithm()

    if not points or frame < 0:
        # Return the unchanged path if there are no points or if frame is negative
        return path,

    if frame >= len(points):
        # If frame is out of bounds, set it to the last frame
        frame = len(points) - 1

    path_data = np.array(points[:frame + 1]).T
    path._offsets3d = path_data

    # Adjust the view angle to rotate the plot
    ax.view_init(elev=45, azim=45)  # You can adjust the 'elev' and 'azim' values

    return path,

# Create animation
animation = FuncAnimation(fig=fig, func=update, frames=len(my_minimization_algorithm()) - 1, interval=30)

animation.save('C:\\Users\\Tommaso\\Desktop\\bproject\\MALIBOO\\gianni.gif')


MovieWriter ffmpeg unavailable; using Pillow instead.
